<a href="https://colab.research.google.com/github/Shona173/codes/blob/main/Comparing_Bounding_brending_and_linear_brending2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install POT
!pip install ffmpeg-python

In [ ]:
import numpy as np
import numpy.random as random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from google.colab import files
from matplotlib import animation
from IPython.display import HTML
import ot

In [ ]:
def clamp(value, min_val, max_val):
    return np.maximum(min_val, np.minimum(value, max_val))

In [ ]:
def sdf_hexagram(p,r=0.5):
    p = p.copy()
    k =np.array[-0.5,0.8660254038,0.5773502692,1.7320508076]
    p=np.abs(p)
    p-=2.0*min(np.dot(k[:2],p),0.0)*k[:2]
    p-=2.0*min(np.dot(k[1::-1],p),0.0)*k[1::-1]
    p-=np.array([clamp(p[:,0],r*k[:,2]),r*k[:,3],r])
    return np.linalg.norm(p)*np.sign(p[1])

In [ ]:
def sdf_rhombus(p,b=[0.7,0.7]):
    p=np.abs(p)
    h=clamp(ndot(b-2.0*p,b)/np.dot(b,b),-1.0,1.0)
    d=np.linalg.norm(p - 0.5 * b * np.array([1.0 - h, 1.0 + h]))
    return d*np.sign(p[:,0]*b[:,1]+p[:,1]*b[:,0]-b[:,0]*b[:,1])

In [ ]:
def gen_grid(resolution, low=-1.0, high=1.0):
    idx = np.linspace(low, high, num=resolution)
    x, y = np.meshgrid(idx, idx)
    V = np.concatenate((x.reshape((-1,1)), y.reshape((-1,1))), 1)

    return V

In [ ]:
def r_union(f1, f2):
    return f1 + f2 + np.sqrt(f1**2 + f2**2)

In [ ]:
def r_intersection(f1, f2):
    return f1 + f2 - np.sqrt(f1**2 + f2**2)

In [ ]:
def linear_morphing(f1, f2, t, x):
    return (1-t)*f1(x) + t*f2(x)

In [ ]:
def space_time_blending(f1, f2, t, x, a0=1.0, a1=1.0, a2=1.0):
    f1x = f1(x)
    f2x = f2(x)

    #f1x_cyl = np.minimum(f1x, -t) # needs R-functions to work properly
    f1x_cyl = r_intersection(f1x, -t)
    #f2x_cyl = np.minimum(f2x, (t-1)) # needs R-functions to work properly
    f2x_cyl = r_intersection(f2x, (t-1))

    disp = a0 / (1.0 + (f1x_cyl/a1)**2 + (f2x_cyl/a2)**2)
    r_uni = r_union(f1x_cyl, f2x_cyl)

    return r_uni + disp

In [ ]:
def bounded_blending(f1, f2, t, x, a0=1.0, a1=1.0, a2=1.0):
  f1x=f1(x)
  f2x=f2(x)
  tx=np.clip(t(x), 0, 1)
  blend=a0*(1-tx)**2+a1*(1-tx)*tx+a2*tx**2
  return (1-blend)*f1x+blend*f2x

In [ ]:
def sample_and_normalize(f, grid, grid_size):
    '''
    Sample f on the grid and normalize it.
    Assume f>0 outside and <0 inside.
    '''
    print(grid.shape)
    fv = f(grid)
    print(fv.shape)

    # >0 inside
    fv = -fv

    # f is the characteristic function for {f>0}
    fv[fv>=0.0] = 1.0
    fv[fv<0.0] = 0.0

    total_sum = np.sum(fv)
    if total_sum > 0:
        fv = fv / total_sum
    else:
        raise ValueError("The sum of the function values is zero; normalization is not possible.")

    # reshape to have the same shape as grid
    fv = fv.reshape(grid_size, grid_size)

    return fv

In [ ]:
grid_size = 64

fig = plt.figure()
x = gen_grid(grid_size,-2.0,2.0)

def animate(t):
    plt.cla()
    out = linear_morphing(sdf_hexagram,sdf_rhombus,np.array([t]), x)
    #levels = np.linspace(-1.0, 1.0, 21)
    #im = plt.contourf(out.reshape(grid_size, grid_size), levels = levels)
    im = plt.contour(out.reshape(grid_size, grid_size), levels = [0.0])
    plt.axis('equal')
    plt.axis("off")
    return im

anim = animation.FuncAnimation(fig, animate, frames=np.linspace(0, 1, 50), interval=50)
anim.save("linear_morphing.mp4", fps=20, writer="ffmpeg")
#anim = animation.FuncAnimation(fig, animate, frames=100, interval=20, blit=True)
#HTML(anim.to_html5_video())
files.download("linear_morphing.mp4")

In [ ]:
grid_size = 64

fig = plt.figure()
x = gen_grid(grid_size, low=-2.0, high=2.0)

def animate(t):
    plt.cla()
    out = space_time_blending(sdf_hexagram,sdf_rhombus,np.array([t]), x, a0=1.0, a1=1.0, a2=1.0)
    im = plt.contour(out.reshape(grid_size, grid_size), levels = [0.0])
    plt.axis('equal')
    plt.axis("off")
    return im
#space time blending
anim = animation.FuncAnimation(fig, animate, frames=np.linspace(0, 1, 50), interval=50)
anim.save("stb_morphing_translated.mp4",fps=20, writer="ffmpeg")
files.download("stb_morphing_translated.mp4")

In [ ]:
grid_size = 64
grid = gen_grid(grid_size,-2.0,2.0)

# f1 and f2 are prob. distribution corresponding to f1 and f2
f1 = sample_and_normalize(sdf_hexagram, grid, grid_size)
f2 = sample_and_normalize(sdf_rhombus, grid, grid_size)

A = np.array([f1,f2])

nb_images = 5
reg = 0.004

v1 = np.array((1, 0))
v2 = np.array((0, 1))

fig, axes = plt.subplots(1, nb_images, figsize=(7, 7))
plt.suptitle("Optimal Trasport")
cm = "Blues"

for i in range(nb_images):
    tx = float(i) / (nb_images - 1)

    weights = (1 - tx) * v1 + tx * v2

    if i == 0:
        axes[i].imshow(f1, cmap=cm)
    elif i == (nb_images - 1):
        axes[i].imshow(f2, cmap=cm)
    else:
        # call to barycenter computation
        axes[i].imshow(
            ot.bregman.convolutional_barycenter2d(A, reg, weights), cmap=cm
        )
    axes[i].axis("off")
#Optimal Trasport
plt.tight_layout()
plt.show()

In [ ]:
grid_size = 64

fig = plt.figure()
x = gen_grid(grid_size, low=-2.0, high=2.0)

def animate(t):
    plt.cla()
    out = bounded_blending(sdf_hexagram,sdf_rhombus,np.array([t]), x, a0=1.0, a1=1.0, a2=1.0)
    im = plt.contour(out.reshape(grid_size, grid_size), levels = [0.0])
    plt.axis('equal')
    plt.axis("off")
    return im
#bounded blending
anim = animation.FuncAnimation(fig, animate, frames=np.linspace(0, 1, 50), interval=50)
anim.save("bd_morphing_translated.mp4",fps=20, writer="ffmpeg")
files.download("bd_morphing_translated.mp4")